# [Synthetic data generation](https://python.langchain.com/docs/use_cases/data_generation)


## Use case

Synthetic data refers to artificially generated data that imitates the characteristics of real data without containing any information from actual individuals or entities. It is typically created through mathematical models, algorithms, or other data generation techniques. Synthetic data can be used for a variety of purposes, including testing, research, and training machine learning models, while preserving privacy and security.

Benefits of Synthetic Data:

1. **Privacy and Security**: No real personal data at risk of breaches.
2. **Data Augmentation**: Expands datasets for machine learning.
3. **Flexibility**: Create specific or rare scenarios.
4. **Cost-effective**: Often cheaper than real-world data collection.
5. **Regulatory Compliance**: Helps navigate strict data protection laws.
6. **Model Robustness**: Can lead to better generalizing AI models.
7. **Rapid Prototyping**: Enables quick testing without real data.
8. **Controlled Experimentation**: Simulate specific conditions.
9. **Access to Data**: Alternative when real data isn't available.

**Note: Despite the benefits, synthetic data should be used carefully, as it may not always capture real-world complexities.**

## Setup
- First, you'll need to have the langchain library installed, along with its dependencies. Since we're using the OpenAI generator chain, we'll install that as well. Since this is an experimental lib, we'll need to include `langchain_experimental` in our installs.
- [Pydantic](https://docs.pydantic.dev/latest/): Data validation library for Python

In [35]:
# %%capture
# !pip install -U langchain langchain_experimental openai
import langchain
langchain.__version__

'0.0.350'

In [20]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.base import SyntheticDataGenerator
from langchain_experimental.tabular_synthetic_data.openai import create_openai_data_generator, OPENAI_TEMPLATE
from langchain_experimental.tabular_synthetic_data.prompts import SYNTHETIC_FEW_SHOT_SUFFIX, SYNTHETIC_FEW_SHOT_PREFIX

In [27]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_organization = os.getenv("OPENAI_API_ORG")

llm = ChatOpenAI(
    temperature=1,
    openai_api_key=openai_api_key,
    # openai_organization=openai_organization
)

In [4]:
SYNTHETIC_FEW_SHOT_SUFFIX, SYNTHETIC_FEW_SHOT_PREFIX

('Now you generate synthetic data about {subject}. Make sure to {extra}:',
 'This is a test about generating synthetic data about {subject}. Examples below:')

## 1. Define Your Data Model
- Every dataset has a structure or a "schema".
- The MedicalBilling class below serves as our schema for the synthetic data.
- By defining this, we're informing our synthetic data generator about the shape and nature of data we expect.

In [5]:
class MedicalBilling(BaseModel):
    patient_id: int
    patient_name: str
    diagnosis_code: str
    procedure_code: str
    total_charge: float
    insurance_claim_amount: float


## 2. Sample Data
To guide the synthetic data generator, it's useful to provide it with a few real-world-like examples. These examples serve as a "seed" - they're representative of the kind of data you want, and the generator will use them to create more data that looks similar.

Here are some fictional medical billing records:

In [6]:
examples = [
    {"example": """Patient ID: 123456, Patient Name: John Doe, Diagnosis Code:
        J20.9, Procedure Code: 99203, Total Charge: $500, Insurance Claim Amount: $350"""},
    {"example": """Patient ID: 789012, Patient Name: Johnson Smith, Diagnosis
        Code: M54.5, Procedure Code: 99213, Total Charge: $150, Insurance Claim Amount: $120"""},
    {"example": """Patient ID: 345678, Patient Name: Emily Stone, Diagnosis Code:
        E11.9, Procedure Code: 99214, Total Charge: $300, Insurance Claim Amount: $250"""},
]

## 3. Craft a Prompt Template
The generator doesn't magically know how to create our data; we need to guide it. We do this by creating a prompt template. This template helps instruct the underlying language model on how to produce synthetic data in the desired format.

In [11]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

prompt_template

FewShotPromptTemplate(input_variables=['extra', 'subject'], examples=[{'example': 'Patient ID: 123456, Patient Name: John Doe, Diagnosis Code:\n        J20.9, Procedure Code: 99203, Total Charge: $500, Insurance Claim Amount: $350'}, {'example': 'Patient ID: 789012, Patient Name: Johnson Smith, Diagnosis\n        Code: M54.5, Procedure Code: 99213, Total Charge: $150, Insurance Claim Amount: $120'}, {'example': 'Patient ID: 345678, Patient Name: Emily Stone, Diagnosis Code:\n        E11.9, Procedure Code: 99214, Total Charge: $300, Insurance Claim Amount: $250'}], example_prompt=PromptTemplate(input_variables=['example'], template='{example}'), suffix='Now you generate synthetic data about {subject}. Make sure to {extra}:', prefix='This is a test about generating synthetic data about {subject}. Examples below:')

The `FewShotPromptTemplate` includes:

- `prefix` and `suffix`: These likely contain guiding context or instructions.
- `examples`: The sample data we defined earlier.
- `input_variables`: These variables ("subject", "extra") are placeholders you can dynamically fill later. For instance, "subject" might be filled with "medical_billing" to guide the model further.
- `example_prompt`: This prompt template is the format we want each example row to take in our prompt.

## 4. Creating the Data Generator
With the schema and the prompt ready, the next step is to create the data generator. This object knows how to communicate with the underlying language model to get synthetic data.

In [28]:
synthetic_data_generator = create_openai_data_generator(
    output_schema=MedicalBilling,
    llm=llm,
    prompt=prompt_template,
)

## 5. Generate Synthetic Data
Finally, let's get our synthetic data!

In [29]:
synthetic_results = synthetic_data_generator.generate(
    subject="medical_billing",
    extra="the name must be chosen at random. Make it something you wouldn't normally choose.",
    runs=10,
)

This command asks the generator to produce 10 synthetic medical billing records. The results are stored in `synthetic_results`. The output will be a list of the MedicalBilling pydantic models.

In [30]:
type(synthetic_results)

list

## 6. Visualize the Generated Synthetic Data

In [31]:
len(synthetic_results)

10

In [32]:
synthetic_results

[MedicalBilling(patient_id=987654, patient_name='Alexandra Bennett', diagnosis_code='I10', procedure_code='99204', total_charge=400.0, insurance_claim_amount=320.0),
 MedicalBilling(patient_id=123456, patient_name='Elijah Thompson', diagnosis_code='A09.9', procedure_code='99215', total_charge=500.0, insurance_claim_amount=400.0),
 MedicalBilling(patient_id=987654, patient_name='Olivia Johnson', diagnosis_code='F32.9', procedure_code='99213', total_charge=250.0, insurance_claim_amount=200.0),
 MedicalBilling(patient_id=987654, patient_name='Emma Martinez', diagnosis_code='K11.7', procedure_code='99203', total_charge=350.0, insurance_claim_amount=280.0),
 MedicalBilling(patient_id=876543, patient_name='Jaxon Reynolds', diagnosis_code='M54.5', procedure_code='99214', total_charge=400.0, insurance_claim_amount=320.0),
 MedicalBilling(patient_id=123456, patient_name='Sophia Thompson', diagnosis_code='E11.9', procedure_code='99212', total_charge=200.0, insurance_claim_amount=160.0),
 Medical

## 7. Converting the synthetic data into Pandas Dataframe

In [33]:
import pandas as pd

# Create a list of dictionaries from the objects
synthetic_data = []
for item in synthetic_results:
    synthetic_data.append({
        'patient_id': item.patient_id,
        'patient_name': item.patient_name,
        'diagnosis_code': item.diagnosis_code,
        'procedure_code': item.procedure_code,
        'total_charge': item.total_charge,
        'insurance_claim_amount': item.insurance_claim_amount
    })

# Create a Pandas DataFrame from the list of dictionaries
synthetic_df = pd.DataFrame(synthetic_data)

# Display the DataFrame
print(type(synthetic_df))
synthetic_df

<class 'pandas.core.frame.DataFrame'>


,patient_id,patient_name,diagnosis_code,procedure_code,total_charge,insurance_claim_amount
0,987654,Alexandra Bennett,I10,99204,400.0,320.0
1,123456,Elijah Thompson,A09.9,99215,500.0,400.0
2,987654,Olivia Johnson,F32.9,99213,250.0,200.0
3,987654,Emma Martinez,K11.7,99203,350.0,280.0
4,876543,Jaxon Reynolds,M54.5,99214,400.0,320.0
5,123456,Sophia Thompson,E11.9,99212,200.0,160.0
6,234567,Xavier Reynolds,R10.4,99202,300.0,240.0
7,987654,Whitney Banks,G47.30,99213,350.0,280.0
8,987654,Oliver Martinez,I10,99214,500.0,400.0
9,123456,Lauren Davis,F32.9,99203,400.0,320.0


In [34]:
synthetic_df.shape

(10, 6)

### Start exploring based on your usecase and use the same approach for real sensitive data. But, be careful, as the synthetic data might not capture the real-world complexities.